In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae, root_mean_squared_error as rmse, r2_score as r2
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [2]:
class BaseExperiment:
    def __init__(self, tickers, start, end, split_date):
        self.tickers = tickers
        self.start = start
        self.end = end
        self.split_date = pd.Timestamp(split_date)

    def load_data(self, ticker):
        df = yf.download(
            ticker,
            start=self.start,
            end=self.end,
            auto_adjust=True,
            progress=False
        )

        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.droplevel(1)

        df.columns.name = None
        df.index.name = "Date"

        # keep exactly what your notebook uses
        df = df[["Open", "High", "Low", "Close", "Volume"]]
        df = df.dropna()

        return df
    @staticmethod
    def build_features(df, block):
        X = pd.DataFrame(index=df.index)
        if block ==0:
            X  = df.drop(columns='Close')

        # B1: close lags
        if block >= 1:
            X["Close_lag1"] = df["Close"].shift(1)
            X["Close_lag2"] = df["Close"].shift(2)

        # B2: returns
        if block >= 2:
            X["ret1"] = df["Close"].pct_change(1)
            X["ret5"] = df["Close"].pct_change(5)

        # B3: trend smoothing
        if block >= 3:
            X["MA5"] = df["Close"].rolling(5).mean()
            X["MA20"] = df["Close"].rolling(20).mean()
            X["EMA5"] = df["Close"].ewm(span=5).mean()
            X["EMA20"] = df["Close"].ewm(span=20).mean()

        # B4: distance from trend
        if block >= 4:
            X["dist_MA5"] = (df["Close"] - X["MA5"]) / X["MA5"]
            X["dist_MA20"] = (df["Close"] - X["MA20"]) / X["MA20"]

        # B6: volume dynamics
        if block >= 5:
            X["vol_chg"] = df["Volume"].pct_change(1)
            X["rel_vol"] = df["Volume"] / df["Volume"].rolling(20).mean()

        return X.dropna()


    def split(self, df):
        train = df[df.index < self.split_date]
        test  = df[df.index >= self.split_date]
        return train, test


In [3]:
class Section1PriceExperiment(BaseExperiment):
    def run(self, models):
        rows = []

        for ticker in self.tickers:
            df = self.load_data(ticker)
            train, test = self.split(df)

            # FEATURES: OHV only
            X_train = train[["Open", "High", "Low", "Volume"]]
            y_train = train["Close"]

            X_test  = test[["Open", "High", "Low", "Volume"]]
            y_test  = test["Close"]

            if len(X_train) == 0 or len(X_test) == 0:
                print(f"Skipping {ticker}: empty train/test")
                continue

            for name, model in models.items():
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                rows.append({
                    "Ticker": ticker,
                    "Model": name,
                    "MAE": mae(y_test, y_pred),
                    "RMSE": rmse(y_test, y_pred),
                    "R2": r2(y_test, y_pred)
                })

        return pd.DataFrame(rows)

class Section2MomentumExperiment(BaseExperiment):
    def run(self, models, m=20):
        rows = []

        for ticker in self.tickers:
            df = self.load_data(ticker)

            # ---- momentum target (difference, not %) ----
            df["momentum"] = df["Close"] - df["Close"].shift(m)
            df = df.dropna()

            train, test = self.split(df)

            # ---- FEATURES: OHLVC (Close included) ----
            feature_cols = ["Open", "High", "Low", "Close", "Volume"]

            X_train = train[feature_cols]
            y_train = train["momentum"]

            X_test  = test[feature_cols]
            y_test_price = test["Close"]

            # guard (same logic as Section 1)
            if len(X_train) == 0 or len(X_test) == 0:
                print(f"Skipping {ticker}: empty train/test")
                continue

            for name, model in models.items():
                # fit on momentum
                model.fit(X_train, y_train)

                # predict momentum
                momentum_pred = model.predict(X_test)

                # reconstruct price forecast
                price_lagged = df["Close"].shift(m).loc[test.index]
                price_pred = price_lagged.values + momentum_pred

                rows.append({
                    "Ticker": ticker,
                    "Model": name,
                    "MAE": mae(y_test_price, price_pred),
                    "RMSE": rmse(y_test_price, price_pred),
                    "R2": r2(y_test_price, price_pred)
                })

        return pd.DataFrame(rows)



In [8]:
models = {
    "LR": LinearRegression(),

    "SVR": Pipeline([
        ("scaler", StandardScaler()),
        ("svr", SVR(C=10, epsilon=0.1, gamma="scale"))
    ]),

    "RF": RandomForestRegressor(
        n_estimators=300,
        max_depth=10,
        n_jobs=-1,
        random_state=42
    ),

    "XGBR": XGBRegressor(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=42
    )
}


In [9]:
blocks = [1,2,3,4,5]
tickers = [
    "RELIANCE.NS",     # Large-cap
    "TCS.NS",          # Large-cap
    "HDFCBANK.NS",     # Large-cap
    "PIDILITIND.NS",   # Mid-cap
    "AUBANK.NS",       # Mid-cap
    "IEX.NS",          # Small-cap
    "MAPMYINDIA.NS",   # Small-cap
    "LTTS.NS",          # Small-cap / mid-tech
    "AAPL",     # Large-cap
    "MSFT",     # Large-cap
    "GOOGL",    # Large-cap
    "AMZN",     # Large-cap
    "NVDA",     # Large-cap
    "CRWD",     # Mid-cap
    "PLTR",     # Mid-cap
    "FSLR",     # Mid-cap
    "CELH",     # Small-cap
    "U",         # Small-cap / mid-tech
    "HSBA.L",   # Large-cap
    "BP.L",     # Large-cap
    "ULVR.L",   # Large-cap
    "AUTO.L",   # Mid-cap
    "JDW.L",    # Mid-cap
    "ASC.L",    # Small-cap
]
start = "2018-03-01"
end = "2025-04-30"
split_date = "2025-04-01"

### Section 1

In [17]:
sec1 = Section1PriceExperiment(
    tickers=tickers,
    start=start,
    end=end,
    split_date=split_date
)

df_sec1 = sec1.run(models=models)
(df_sec1)


,Ticker,Model,MAE,RMSE,R2
0,RELIANCE.NS,LR,4.824803,6.753846,0.987270
1,RELIANCE.NS,SVR,19.098589,29.224895,0.761646
2,RELIANCE.NS,RF,8.334754,11.282069,0.964478
3,RELIANCE.NS,XGBR,9.982347,13.139380,0.951820
4,TCS.NS,LR,15.524450,20.662972,0.954958
...,...,...,...,...,...
91,JDW.L,XGBR,5.615568,6.989245,0.958895
92,ASC.L,LR,6.010040,7.481143,0.871683
93,ASC.L,SVR,121.585667,127.999438,-36.563452
94,ASC.L,RF,9.062152,10.550731,0.744780


In [22]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.3 MB/s eta 0:00:00


In [23]:
df_pivoted = df_sec1.pivot(index='Ticker', columns='Model', values=['MAE', 'RMSE', 'R2'])
df_pivoted = df_pivoted.reindex(columns=['MAE', 'RMSE', 'R2'], level=0)
styled_df = df_pivoted.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f7f7f7')]},
    {'selector': 'td', 'props': [('text-align', 'center')]} # Optional: centers the data values too
])

styled_df.to_excel("Stock_Metrics1.xlsx", engine='xlsxwriter')
styled_df

### section 2

In [25]:
sec2 = Section2MomentumExperiment(
    tickers=tickers,
    start=start,
    end=end,
    split_date=split_date
)

df_sec2 = sec2.run(
    models=models,
    m=20
)


df_pivoted = df_sec2.pivot(index='Ticker', columns='Model', values=['MAE', 'RMSE', 'R2'])
df_pivoted = df_pivoted.reindex(columns=['MAE', 'RMSE', 'R2'], level=0)
styled_df = df_pivoted.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f7f7f7')]},
    {'selector': 'td', 'props': [('text-align', 'center')]} # Optional: centers the data values too
])

styled_df.to_excel("Stock_Metrics2.xlsx", engine='xlsxwriter')
styled_df

### Section 3

In [4]:
def build_feature_block(df, block):
    X = pd.DataFrame(index=df.index)
    # B1: Close lags
    if block >= 1:
        X["Close_lag1"] = df["Close"].shift(1)
        X["Close_lag2"] = df["Close"].shift(2)

    # B2: Returns
    if block >= 2:
        X["ret1"] = df["Close"].pct_change(1)
        X["ret5"] = df["Close"].pct_change(5)

    # B3: MA + EMA
    if block >= 3:
        X["MA5"]  = df["Close"].rolling(5).mean()
        X["MA20"] = df["Close"].rolling(20).mean()
        X["EMA5"]  = df["Close"].ewm(span=5).mean()
        X["EMA20"] = df["Close"].ewm(span=20).mean()

    # B4: Distance from trend
    if block >= 4:
        X["dist_MA5"]  = (df["Close"] - X["MA5"]) / X["MA5"]
        X["dist_MA20"] = (df["Close"] - X["MA20"]) / X["MA20"]

    # B5: Volume dynamics
    if block >= 5:
        X["vol_chg"] = df["Volume"].pct_change(1)
        X["rel_vol"] = df["Volume"] / df["Volume"].rolling(20).mean()

    X.replace([np.inf, -np.inf], 0, inplace=True)

    return X.dropna()


In [5]:
class Section3FeatureBlockExperiment(BaseExperiment):
    def run(self, models, blocks):
        rows = []

        for ticker in self.tickers:
            df = self.load_data(ticker)

            for block in blocks:
                X = build_feature_block(df, block)
                y = df.loc[X.index, "Close"]

                train, test = self.split(df)

                X_train = X.loc[train.index.intersection(X.index)]
                y_train = y.loc[X_train.index]

                X_test  = X.loc[test.index.intersection(X.index)]
                y_test  = y.loc[X_test.index]

                if len(X_train) == 0 or len(X_test) == 0:
                    print(f"Skipping {ticker} | B{block}: insufficient data")
                    continue

                for name, model in models.items():
                    model.fit(X_train, y_train)
                    y_pred = model.predict(X_test)

                    rows.append({
                        "Ticker": ticker,
                        "Model": name,
                        "Block": f"B{block}",
                        "MAE": mae(y_test, y_pred),
                        "RMSE": rmse(y_test, y_pred),
                        "R2": r2(y_test, y_pred)
                    })

        return pd.DataFrame(rows)


In [10]:
sec3 = Section3FeatureBlockExperiment(
    tickers=tickers,
    start=start,
    end=end,
    split_date=split_date
)

df_sec3 = sec3.run(
    models=models,
    blocks=[1, 2, 3, 4, 5]
)

df_pivoted = df_sec3.pivot(index='Ticker', columns='Model', values=['MAE', 'RMSE', 'R2'])
df_pivoted = df_pivoted.reindex(columns=['MAE', 'RMSE', 'R2'], level=0)
styled_df = df_pivoted.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f7f7f7')]},
    {'selector': 'td', 'props': [('text-align', 'center')]} # Optional: centers the data values too
])

styled_df.to_excel("Stock_Metrics3.xlsx", engine='xlsxwriter')
styled_df

ValueError: Index contains duplicate entries, cannot reshape

In [13]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.6 MB/s eta 0:00:00


In [14]:
df_melted = df_sec3.melt(id_vars=['Ticker', 'Block', 'Model'],
                    value_vars=['MAE', 'RMSE', 'R2'],
                    var_name='Metric', value_name='Score')

final_table = df_melted.pivot_table(index='Ticker',
                                   columns=['Block', 'Metric', 'Model'],
                                   values='Score')
# Apply styling to center column headers and data
styled_table = final_table.style.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]}
]).set_properties(**{'text-align': 'center'})

# Display the styled table
styled_table.to_excel("Stock_Metrics3.xlsx", engine='xlsxwriter')
styled_table